In [30]:
import os
import ase
import yaml
from ase.build import fcc111, fcc100, fcc110
from moleidoscope.visualize import show
from moleidoscope.linker import Linker
from moleidoscope.geo.vector import align
from xyz2cif import write_cif

In [31]:
def ase_to_linker(ase_atoms):
    """ Convert ASE atoms object to Moleidoscope linker object """
    mol = Linker()
    mol.atom_names = list(ase_atoms.get_chemical_symbols())
    mol.atom_coors = list(ase_atoms.get_positions())
    return mol

def read_yaml(yaml_file):
    with open(yaml_file, 'r') as yf:
        data = yaml.load(yf)
    return data

## Read surfaces and molecules

In [32]:
mol_dir = '/home/kutay/Documents/git/Nanocar/molecules/opt'
molecules_info = read_yaml('molecules.yaml')
surfaces_info = read_yaml('surfaces.yaml')

## Set parameters

In [33]:
molecule = 'BtPHD'
metal = 'Cu'

mol = molecules_info[molecule]
surf = surfaces_info[metal]
z_box = 50                    # Length of simulation box in z-direction (A)
distance_from_surface = 5     # Distance btw molecule and surface (A)

## Metal Slab

In [34]:
ase_slab = fcc110(metal, a=surf['a'], size=surf['size'], vacuum=surf['vacuum'])
l_slab = ase_to_linker(ase_slab)
print(ase_slab.cell)

[[ 35.97         0.           0.        ]
 [  0.          38.15194638   0.        ]
 [  0.           0.           5.08692618]]


In [35]:
# show(cu110)

### Aligning the molecule (*xy-plane*)
- Each molecule has a list of two atoms to be aligned to *x-axis* and *y-axis*
- First we align the vector formed by the first two atoms to *x-axis*.
- Then we use the second ector formed by the second set of atoms and align that vector to *y-axis.*
- By aligning these vectors the molecule is now aligned to *xy-plane*.

In [36]:
l_mol = Linker(read=os.path.join(mol_dir, '%s.xyz' % molecule))

l_mol.vector = l_mol.get_vector(mol['x'][0], mol['x'][1])
l_mol = l_mol.align([1, 0, 0])
l_mol.center([0, 0, 0])

l_mol.vector = l_mol.get_vector(mol['y'][0], mol['y'][1])
l_mol = l_mol.align([0, 1, 0])
l_mol.center([0, 0, 0])

In [37]:
show(l_mol)

The installed widget Javascript is the wrong version.


### Placing aligned molecule on metal surface
The aligned molecule will be place 5 Å above the metal surface.

In [38]:
slab_center = l_slab.get_center()
l_mol.center(slab_center)
l_mol.translate([0, 0, distance_from_surface + ase_slab.cell[2][2] / 2])
mol_slab = l_slab.join(l_mol)

In [39]:
show(mol_slab)

The installed widget Javascript is the wrong version.


In [17]:
# mol_slab.name='%s_%s110' % (molecule, metal)
# mol_slab.save(file_format='xyz')

## Create cif file

In [20]:
# 'BTW_FF', 'Dreiding', 'UFF', 'UFF4MOF', and 'Dubbeldam'
force_field = 'UFF'
sim_box = [ase_slab.cell[0][0], ase_slab.cell[1][1], z_box, 90, 90, 90]
cif_file = '%s_%s.cif' % (mol_slab.name, force_field)
write_cif(cif_file, mol_slab.atom_names, mol_slab.atom_coors, header=cif_file, cell=sim_box, fractional=True)

## Create Lammps simulation files

In [21]:
import os
from thermof import Simulation
from thermof import Parameters

In [22]:
simpar = Parameters()
sim = Simulation(mof=cif_file, parameters=simpar)
mof_name = os.path.splitext(os.path.basename(cif_file))[0]
sim.simdir = os.path.join(os.path.dirname(cif_file), mof_name)

sim.parameters.lammps['force_field'] = force_field
sim.parameters.lammps['mol_ff'] = force_field
sim.parameters.thermof['fix'] = ['NVT', 'NVE']
sim.parameters.thermof['temperature'] = 200
sim.parameters.thermof['thermo_style'] = ['step', 'temp', 'press', 'pe', 'etotal', 'epair', 'emol', 'evdwl', 'ecoul']
sim.parameters.job['scheduler'] = 'slurm'
sim.parameters.job['cluster'] = 'smp'
sim.parameters.job['nodes'] = 1
sim.parameters.job['ppn'] = 24

In [23]:
sim.initialize()

I. Writing Lammps input and data files...
No bonds reported in cif file - computing bonding..


Files created! -> HtBDC_Cu110_UFF
II. Updating Lammps input file -> HtBDC_Cu110_UFF/in.HtBDC_Cu110_UFF
Adding fixes: NVT | NVE
Updating simulation parameters...
III. Writing slurm job submission file -> HtBDC_Cu110_UFF/job.HtBDC_Cu110_UFF
Simulation parameters saved -> HtBDC_Cu110_UFF/simpar.yaml
Done!


In [2]:
import yaml

In [10]:
ll = {'mol1': {'asd': 123, 've': 'asd', 'x': [1, 2]}, 'mol2': {'asd': 5432, 'jer': 'das', 'y': [3, 4]}}

In [11]:
with open('test.yaml', 'w') as yf:
    yaml.dump(ll, yf)